In [1]:
with open('../data/wine_links_0_to_6.txt', 'r') as a, open('../data/wine_links_6_to_7.txt', 'r') as b, open('../data/wine_links_7_to_8.txt', 'r') as c, open('../data/wine_links_8_to_9.txt', 'r') as d, open('../data/wine_links_9_to_11.txt', 'r') as e, open('../data/wine_links_11_to_12.txt', 'r') as f, open('../data/wine_links_12_to_15.txt', 'r') as g, open('../data/wine_links_15_to_16.txt', 'r') as h, open('../data/wine_links_16_to_20.txt', 'r') as i, open('../data/wine_links_20_to_21.txt', 'r') as j, open('../data/wine_links_21_to_30.txt', 'r') as k, open('../data/wine_links_30_to_60.txt', 'r') as l, open('../data/wine_links_60_to_500plus.txt', 'r') as m:
    
    links = a.readlines()+b.readlines()+c.readlines()+d.readlines()+e.readlines()+f.readlines()+g.readlines()+h.readlines()+i.readlines()+j.readlines()+k.readlines()+l.readlines()+m.readlines()

In [2]:
len(links)

12588

In [3]:
links = [link.replace("\n", "") for link in links]

In [4]:
links = [link for link in links if "https://www.vivino.com" in link]

In [5]:
from selenium import webdriver

from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import time

In [6]:
#driver configuration
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False    # si True, no aperece la ventana (headless=no visible)
opciones.add_argument('--start-maximized')         # comienza maximizado
#opciones.add_argument('user-data-dir=selenium')    # mantiene las cookies
#opciones.add_extension('driver_folder/adblock.crx')       # adblocker
opciones.add_argument('--incognito')

In [7]:
PATH=ChromeDriverManager().install()

driver = webdriver.Chrome(PATH,options = opciones)

C:\Users\eloyl\AppData\Local\Temp\ipykernel_8200\2256630989.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH,options = opciones)


In [8]:
complete_data = []

In [71]:
def data_from_links(url):
    try:
        # start driver 
        driver=webdriver.Chrome(PATH, options= opciones)
        driver.get(url)

        time.sleep(4)
    
        #cookies banner selection
        btn_cookie = driver.find_element(By.XPATH, '//*[@id="cookie-notice-container"]/div/button')
        #cookie banner click
        btn_cookie.click()
    
        time.sleep(4)
        
        #end page
        driver.find_element(By.XPATH,'//body').send_keys(Keys.END)
        time.sleep(1)
        driver.find_element(By.XPATH,'//body').send_keys(Keys.END)
        time.sleep(1)
        driver.find_element(By.XPATH,'//body').send_keys(Keys.END)
        time.sleep(1)
        driver.find_element(By.XPATH,'//body').send_keys(Keys.END)
        time.sleep(1)
        driver.find_element(By.XPATH,'//body').send_keys(Keys.END)
        time.sleep(1)
        
        #data
        top_web = driver.find_element(By.XPATH, '/html/body/div[2]/div[5]/div/div/div[2]')
        wine = [i.text for i in top_web.find_elements(By.TAG_NAME, "a")]

        #save data to variables
        
        bodega = driver.find_element(By.CLASS_NAME, "row.header.breadCrumbs").text.split("\n")[0]
        
        nombre = driver.find_element(By.CLASS_NAME, "row.header.breadCrumbs").text.split("\n")[1]
        
        valoracion = [i for i in wine if "valoraciones" in i][0].split("\n")[0]
        
        num_valoraciones = [i for i in wine if "valoraciones" in i][0].split("\n")[1].split(" ")[0]
        
        precio = [i for i in driver.find_element(By.CLASS_NAME, "vintageSelection").text.split("\n") if "€" in i][0][1:]
        if len(precio) > 6:
            precio = None
         
        try:
            año = url.split("?")[1].split("&")[0][-4:]
        except:
            año = None


        zona = [i for i in [i.text for i in driver.find_elements(By.CSS_SELECTOR, "tr")] if "Región" in i][0].split("\n")[1]
        
        tipo = [i for i in wine if "Vino" in i][0]
        
        uva= [i for i in [i.text for i in driver.find_elements(By.CSS_SELECTOR, "tr")] if "Uvas" in i][0].split("\n")[1]
        #save data to a dictionary and export
        complete_data = {
    
        'winery': bodega,
        'wine': nombre,
        'year': año,
        'rating': valoracion,
        'num_review': num_valoraciones,
        'region': zona,
        'price': precio,
        'type': tipo,
        'grapes' : uva
    
        } 
    
        return complete_data
    except:
        return url

In [72]:
from joblib import Parallel, delayed

import pandas as pd

from tqdm.notebook import tqdm 

In [73]:
len(links)

12213

In [12]:
links = list(set(links))

In [13]:
len(links)

12213

In [79]:
%%time
data = []

data = [Parallel(n_jobs=12, verbose=True)(delayed(data_from_links)(url) for url in tqdm(link))]

  0%|          | 0/100 [00:00<?, ?it/s]

[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:  1.3min


CPU times: total: 234 ms
Wall time: 3min 31s


[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:  3.5min finished


In [82]:
data = data[0]

In [89]:
data

[{'winery': 'Bodegas El Sequé',
  'wine': 'El Sequé 2017',
  'year': '2017',
  'rating': '4,1',
  'num_review': '421',
  'region': 'España / Valencia / Alicante',
  'price': '20,90',
  'type': 'Vino tinto',
  'grapes': '100 % de Monastrell'},
 {'winery': 'De Muller',
  'wine': 'Syrah 2019',
  'year': '2019',
  'rating': '3,7',
  'num_review': '305',
  'region': 'España / Cataluña / Tarragona',
  'price': '7,85',
  'type': 'Vino tinto',
  'grapes': '100 % de Shiraz/Syrah'},
 {'winery': 'Javier Sanz Viticultor',
  'wine': 'Sauvignon Blanc 2021',
  'year': '2021',
  'rating': '4,1',
  'num_review': '92',
  'region': 'España / Castilla y León / Rueda',
  'price': '9,30',
  'type': 'Vino blanco',
  'grapes': '100 % de Sauvignon Blanc'},
 {'winery': 'Aalto',
  'wine': 'PS (Pagos Seleccionados) Ribera del Duero 2017',
  'year': '2017',
  'rating': '4,6',
  'num_review': '3473',
  'region': 'España / Castilla y León / Ribera del Duero',
  'price': '80,30',
  'type': 'Vino tinto',
  'grapes': '

In [ ]:
df_wines = pd.DataFrame(data)

In [ ]:
df_wines.to_csv('../data/allwines.csv',index=False, header=True)